In [ ]:
import nltk
from collections import defaultdict
import re
import pickle as pkl
import time

##### 1. Building a Conversion Dictionary

In [ ]:
dict1 = defaultdict(list)#word2pos
dict2 = defaultdict(list)#pos2word

In [ ]:
train_data = read_txt("wmt14_data/train.tgt")
pos_train = read_txt("wmt14_data/pos/train.tgt")

In [ ]:
for i in range(len(train_data)):
    line1 = train_data[i]
    line2 = pos_train[i]
    for j in range(len(line1)):
        w1 = line1[j]
        w2 = line2[j]
        if w2 not in dict1[w1]:
            dict1[w1].append(w2)
        if w1 not in dict2[w2]:
            dict2[w2].append(w1)

In [ ]:
pkl.dump(dict1,open("wmt14_data/train_word2pos.pkl","wb"))
pkl.dump(dict2,open("wmt14_data/train_pos2word.pkl","wb"))

##### 2. POS dict to BPE dict idx

In [ ]:
# please prepare a word2idx dict from cmlm model first, forms like {'<s>': 0, '<pad>': 1, '</s>': 2, ....} type dict()
w2idx = pkl.load(open("wmt14_data/wmt14_cmlm_word2idx_dict.pkl","rb"))  
w2idx = dict(sorted(w2idx.items(), key=lambda item: item[1], reverse=False))

In [ ]:
idx2w = pkl.load(open("wmt14_data/train_pos2word.pkl","rb"))

In [ ]:
# First, we bulid a Conversion Dictionary with POS tag as the key, while the value is a
# set of words that can be assigned to this POS tag. we then convert the value a binarized vector (length=vocabulary size)
import torch
pos_vocab = defaultdict(list)
for key in idx2w.keys():
    words = idx2w[key]
    words2 = list(w2idx.keys())
    one_hot = []
    for i in range(length):
        if words2[i] in words:
            one_hot.append(1)
        else:
            one_hot.append(0)
    pos_vocab[key] = torch.FloatTensor(one_hot)

In [ ]:
pos_vocab["<mask>"] = torch.FloatTensor([0,0,0,0,1]+[0 for i in range(length-5)])
pos_vocab["<unk>"] = torch.FloatTensor([0,0,0,1]+[0 for i in range(length-4)])
pos_vocab["</s>"] = torch.FloatTensor([0,0,1]+[0 for i in range(length-3)])
pos_vocab["<pad>"] = torch.FloatTensor([0,1]+[0 for i in range(length-2)])
pos_vocab["<s>"] = torch.FloatTensor([1]+[0 for i in range(length-1)])

In [ ]:
for key in pos_vocab.keys():
    print(key,sum(pos_vocab[key])*100/len(pos_vocab[key]))

In [ ]:
pkl.dump(pos_vocab,open("wmt14_data/pos2vocab_onehot_cmlm.pkl","wb"))

In [ ]:
pos_vocab = pkl.load(open("wmt14_data/pos2vocab_onehot_cmlm.pkl","rb"))

In [ ]:
pos_test = read_txt("") # please #fill with the user dir# in step3

In [ ]:
#generate a binarized mask matrix for the test set
pos_metric = defaultdict(list)
for i in range(len(pos_test)):
    line = pos_test[i]
    pos = []
    for j in range(len(line)):
        pos.append(pos_vocab[line[j]])
    pos.append(pos_vocab["</s>"])
    pos_metric[i]=pos

In [ ]:
pkl.dump(pos_metric,open("wmt14_data/wmt14_test_posmask_for_cmlm.pkl","wb"))